In [1]:
from desdeo_emo.population import SurrogatePopulation
from desdeo_emo.recombination import SinglePoint_Xover
from desdeo_problem.testproblems import test_problem_builder

In [3]:
problem = test_problem_builder("DTLZ2", 5, 3)

